<a href="https://colab.research.google.com/github/AnaBeatrizMassuh/Competi-o/blob/main/Competi%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import numpy as np
import nltk
from nltk.corpus import reuters
import math
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.corpus import wordnet as wn


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
def pre_processing(text):
  stopwords_nltk = stopwords.words('portuguese')
  wn_lemmas = set(wn.all_lemma_names())
  
  words = re.sub(r'[\s!%*~\^´`=+<\[\]?&$:;@#.0-9()\/\"\'_-]+|[\s]*[\\]u[a-z0-9]{4}[\s]*|[\s]*http[^\s]+[\s]*', " ", text.lower())
  words = re.sub(r'\brt\b', "", words)
  words = re.sub(r'[\\]+', " ", words)
  words = words.strip()
  
  phrase = ""
  words = words.split()

  for x in words:
    if x not in stopwords_nltk and x in wn_lemmas:
      phrase = phrase + " " + x
        
  return phrase.strip()

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/viniciusrpb/cic0269_natural_language_processing/main/datasets/corpora/test.csv", sep=',', names=['id','text'])
df

,id,text
0,Id,text
1,5000,O Naoum Hotel está lozalizado no centro comerc...
2,5001,Ótima localização para quem vai tirar o visto ...
3,5002,"Esta é a segunda vez que fico neste hotel, dev..."
4,5003,"Boa localização com café da manhã razoável, To..."
...,...,...
396,5395,A localização o faz um ótimo hotel para se est...
397,5396,Há uma preocupação com a limpeza a com o confo...
398,5397,"O Mercure Lider, é uma boa opção à negócios, p..."
399,5398,"O Hotel é bem localizado, muito perto das prin..."


In [ ]:
df['clean_text'] = df['text'].str.lower()
df

,id,text,clean_text
0,Id,text,text
1,5000,O Naoum Hotel está lozalizado no centro comerc...,o naoum hotel está lozalizado no centro comerc...
2,5001,Ótima localização para quem vai tirar o visto ...,ótima localização para quem vai tirar o visto ...
3,5002,"Esta é a segunda vez que fico neste hotel, dev...","esta é a segunda vez que fico neste hotel, dev..."
4,5003,"Boa localização com café da manhã razoável, To...","boa localização com café da manhã razoável, to..."
...,...,...,...
396,5395,A localização o faz um ótimo hotel para se est...,a localização o faz um ótimo hotel para se est...
397,5396,Há uma preocupação com a limpeza a com o confo...,há uma preocupação com a limpeza a com o confo...
398,5397,"O Mercure Lider, é uma boa opção à negócios, p...","o mercure lider, é uma boa opção à negócios, p..."
399,5398,"O Hotel é bem localizado, muito perto das prin...","o hotel é bem localizado, muito perto das prin..."


In [ ]:
stopwords_nltk = stopwords.words('portuguese')
df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_nltk)]))
df

,id,text,clean_text
0,Id,text,text
1,5000,O Naoum Hotel está lozalizado no centro comerc...,naoum hotel lozalizado centro comercial brasíl...
2,5001,Ótima localização para quem vai tirar o visto ...,ótima localização vai tirar visto americano. d...
3,5002,"Esta é a segunda vez que fico neste hotel, dev...","segunda vez fico neste hotel, devido preço óti..."
4,5003,"Boa localização com café da manhã razoável, To...","boa localização café manhã razoável, todo hote..."
...,...,...,...
396,5395,A localização o faz um ótimo hotel para se est...,"localização faz ótimo hotel estar, afinal cont..."
397,5396,Há uma preocupação com a limpeza a com o confo...,preocupação limpeza conforto hóspede. todos eq...
398,5397,"O Mercure Lider, é uma boa opção à negócios, p...","mercure lider, boa opção negócios, pois bem lo..."
399,5398,"O Hotel é bem localizado, muito perto das prin...","hotel bem localizado, perto principais atraçõe..."


In [ ]:
corpus = []
for row in df['clean_text']:
  corpus.append(row)

corpus

In [ ]:
phrase = " ".join(corpus)
tokens = phrase.split()
vocab = sorted(set(tokens))

In [ ]:
bow_ponderado = {}
N = {} # quantidade de termos em cada documento

for i,doc in enumerate(corpus):
    bow_ponderado[i] = dict()
    N[i] = len(doc.split())
    for word in doc.split():
        if word in bow_ponderado[i]:
            bow_ponderado[i][word] += 1
        else:
            bow_ponderado[i][word] = 1

for doc in bow_ponderado:
    for word in bow_ponderado[doc]:
        bow_ponderado[doc][word] /= N[doc]

In [ ]:
bow_contagem = {}

for i,doc in enumerate(corpus):
    bow_contagem[i] = dict()
    for word in doc.split():
        if word in bow_contagem[i]:
            bow_contagem[i][word] += 1
        else:
            bow_contagem[i][word] = 1

In [ ]:
idf = {}
for word in vocab:
  idf[word] = 0
  #contabilizar em quantos documentos a palavra word aparece
  for doc in bow_contagem:
    if word in bow_contagem[doc]:
      idf[word] += 1

for word in idf:
  if(idf[word] == 0): continue
  idf[word] = math.log(len(corpus)/idf[word])

In [ ]:
bow_tfidf = {}

for doc in bow_ponderado:
  bow_tfidf[doc] = dict()
  for word in bow_ponderado[doc]:
    bow_tfidf[doc][word] = bow_ponderado[doc][word]*idf[word]

In [ ]:
def cosine_dissimilarity(vector1, vector2):
  prod_interno = 0

  for i, value in enumerate(vector1):
    prod_interno += value*vector2[i]

  norma_vector1 = 0

  for x in vector1:
    norma_vector1 += x*x
  
  norma_vector1 = norma_vector1**0.5

  #norma_vector1 = math.sqrt(sum([x**2 for x in vector1]))

  norma_vector2 = 0

  for x in vector1:
    norma_vector2 += x*x
  
  norma_vector2 = norma_vector2**0.5

  return prod_interno/(norma_vector1*norma_vector2)